In [1]:
#start

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
16,application_1726718440129_0018,pyspark,idle,Link,Link,user_n-renukumar_rajendran_draup_com,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.addPyFile("s3://jobs-emr-etl-config/jobs_run/prod/cc/ML_ETL_Config.py")
from ML_ETL_Config import *
utils_module = MlEtlUtils(spark, sc)
to_import = [name for name in dir(utils_module) if not (name.startswith('_') | (name=="spark") | (name=="sc"))]
globals().update({name: getattr(utils_module, name) for name in to_import})
from pyspark.sql.functions import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

404
404
/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gateway.draup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gateway.draup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(

# Reading CB data

In [3]:
company=read_superadmin_collection('crunchbase_daily_sample','crunchbase_merged_collection').selectExpr('cb_markets','company_type','permalink','crunchbase_overview.founded_on','crunchbase_overview.name','crunchbase_overview.legal_name','crunchbase_overview.description','crunchbase_overview.short_description','crunchbase_overview.employee_count','funding_info.num_funding_rounds','funding_rounds.funding_rounds_name','websites.cb_url','websites.homepage_url','websites.domain_url','websites.email','websites.facebook_url','websites.linkedin_url','websites.twitter_url','websites.logo_url','company_markets.category as category','company_markets.category_groups as category_groups','company_industry','markets','headquarters.organizations_location','headquarters.organizations_msa','headquarters.country_code','headquarters.address','headquarters.city','headquarters.regoin','headquarters.state_code','headquarters.organizations_synon_location','ipo_list.stock_exchange_symbol','ipo_list.stock_symbol','investments.investor_info.investor_type')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
company_str_data=company.select([col(c).cast('string') for c in company.columns]).drop_duplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Permalink Input

In [10]:
df=spark.read.csv("s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/output/Linkedin_Details_use_domain_URL_03-09-2024.csv",header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['_c0', 'Updated Draup Name', 'Cleaned URL', 'domain_url', 'linkedin_Name', 'Linkedin_link', 'Linkedin_website', 'permalink', 'industry', 'headquarter', 'about', 'company_size', 'employee_count', 'year_established', 'logo_url', 'account_type', 'date_updated', 'Linkedin_domain']

In [12]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1121

In [11]:
main_df=df.join(company_str_data,df['CB URL']==company_str_data["permalink"],how="left").distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
main_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Company Name', 'CB URL', 'cb_markets', 'company_type', 'permalink', 'founded_on', 'name', 'legal_name', 'description', 'short_description', 'employee_count', 'num_funding_rounds', 'funding_rounds_name', 'cb_url', 'homepage_url', 'domain_url', 'email', 'facebook_url', 'linkedin_url', 'twitter_url', 'logo_url', 'category', 'category_groups', 'company_industry', 'markets', 'organizations_location', 'organizations_msa', 'country_code', 'address', 'city', 'regoin', 'state_code', 'organizations_synon_location', 'stock_exchange_symbol', 'stock_symbol', 'investor_type']

In [13]:
main_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

390

In [ ]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=main_df.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/CB_Details_use_permalinks_data_20-08-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Domain input

In [5]:
df=spark.read.csv("s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/Input/linkedin_CB_19-09-24.csv",header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Company Name', 'Cleaned Company Url']

In [7]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

876

In [8]:
#cleaning domain urls
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Define a UDF to extract the domain name from a URL https://www.linkedin.com/company/draupplatform/
def extract_parma(url):
    try:
        if "www." in url:
            url = url.split("www.")[1].split('/')[0]
        elif "//"in url:
            url = url.split("//")[1].split('/')[0]
        return url
    except:
        return url

# Register the UDF with Spark
extract_parma_udf = udf(extract_parma, StringType())

df1 = df.withColumn('cleaned_domain', extract_parma_udf(df['Cleaned Company Url']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
fin_df=df1.join(company_str_data,df1["cleaned_domain"]==company_str_data["domain_url"],how="left").distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
fin_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Company Name', 'Cleaned Company Url', 'cleaned_domain', 'cb_markets', 'company_type', 'permalink', 'founded_on', 'name', 'legal_name', 'description', 'short_description', 'employee_count', 'num_funding_rounds', 'funding_rounds_name', 'cb_url', 'homepage_url', 'domain_url', 'email', 'facebook_url', 'linkedin_url', 'twitter_url', 'logo_url', 'category', 'category_groups', 'company_industry', 'markets', 'organizations_location', 'organizations_msa', 'country_code', 'address', 'city', 'regoin', 'state_code', 'organizations_synon_location', 'stock_exchange_symbol', 'stock_symbol', 'investor_type']

In [11]:
fin_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

896

In [12]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=fin_df.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/CB_Details_use_domain_19-09-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'BHADG921BH8YNRJH', 'HostId': '6Eb4CxFmsP898IkD7Mbj6KGWjGlQ7vNFI44+s6iJiUdY6J+DkqZzvnAt0bekb5OVoXB1t+GpDcs=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '6Eb4CxFmsP898IkD7Mbj6KGWjGlQ7vNFI44+s6iJiUdY6J+DkqZzvnAt0bekb5OVoXB1t+GpDcs=', 'x-amz-request-id': 'BHADG921BH8YNRJH', 'date': 'Thu, 19 Sep 2024 05:40:44 GMT', 'x-amz-expiration': 'expiry-date="Wed, 25 Sep 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"1f7ed827b67db2fad4e4453fdcff66ac"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Wed, 25 Sep 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"1f7ed827b67db2fad4e4453fdcff66ac"', 'ServerSideEncryption': 'AES256'}

# Name match

In [13]:
df=spark.read.csv("s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/Input/Linkedin_domin_input_17_09_2024.csv",header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['S.No', 'Company Name', 'Website']

In [17]:
df1=df.withColumn('lower_name',lower(col('Company Name')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
company_str_data1=company_str_data.withColumn('lower_name_',lower(col('name')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
fin_df=df1.join(company_str_data1,df1["lower_name"]==company_str_data1["lower_name_"],how="left").drop("lower_name_","lower_name")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
fin_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['S.No', 'Company Name', 'Website', 'cb_markets', 'company_type', 'permalink', 'founded_on', 'name', 'legal_name', 'description', 'short_description', 'employee_count', 'num_funding_rounds', 'funding_rounds_name', 'cb_url', 'homepage_url', 'domain_url', 'email', 'facebook_url', 'linkedin_url', 'twitter_url', 'logo_url', 'category', 'category_groups', 'company_industry', 'markets', 'organizations_location', 'organizations_msa', 'country_code', 'address', 'city', 'regoin', 'state_code', 'organizations_synon_location', 'stock_exchange_symbol', 'stock_symbol', 'investor_type']

In [21]:
fin_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

122

In [22]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=fin_df.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/CB_Details_using_name_17-09-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '207NKA2ZDGN7NFWQ', 'HostId': 'arOL9loRSriQ9zoBUf18GEzftDln+/reBpnGudS+L4ZIUUedKk4fN5Q95NrAooE2AZBS4TsB2o4=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'arOL9loRSriQ9zoBUf18GEzftDln+/reBpnGudS+L4ZIUUedKk4fN5Q95NrAooE2AZBS4TsB2o4=', 'x-amz-request-id': '207NKA2ZDGN7NFWQ', 'date': 'Tue, 17 Sep 2024 09:49:37 GMT', 'x-amz-expiration': 'expiry-date="Mon, 23 Sep 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"b308711192c79f9297b28f4ec65d3381"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Mon, 23 Sep 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"b308711192c79f9297b28f4ec65d3381"', 'ServerSideEncryption': 'AES256'}

In [14]:
company=read_superadmin_collection('crunchbase_daily_sample','crunchbase_merged_collection').selectExpr('cb_markets','company_type','permalink','crunchbase_overview.founded_on','crunchbase_overview.name','crunchbase_overview.legal_name','crunchbase_overview.description','crunchbase_overview.short_description','websites.cb_url','websites.homepage_url','websites.domain_url','websites.facebook_url','websites.linkedin_url','websites.twitter_url','company_industry','markets','headquarters.organizations_location','headquarters.organizations_msa','headquarters.country_code')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
company.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['cb_markets', 'company_type', 'permalink', 'founded_on', 'name', 'legal_name', 'description', 'short_description', 'cb_url', 'homepage_url', 'domain_url', 'facebook_url', 'linkedin_url', 'twitter_url', 'company_industry', 'markets', 'organizations_location', 'organizations_msa', 'country_code']

In [18]:
company_str_data=company.select([col(c).cast('string') for c in company.columns])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
company.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3431489

In [19]:
split_ratios = [1/4] * 4  # Equal split ratios for 9 parts
split_datasets = company_str_data.randomSplit(split_ratios, seed=42)

# Access each split DataFrame
split_df1 = split_datasets[0]
split_df2 = split_datasets[1]
split_df3 = split_datasets[2]
split_df4 = split_datasets[3]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
split_df1.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

858097

In [20]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=split_df1.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/Avinash/Output/CB_data_dump_set1_14-05-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'MY74Z7CNEWGXC8T0', 'HostId': 'ejy8473/GQUnNIwpjTZPFY/SSnP4a4zA16GcLXHjKG0MZZJY4ZoPZ2ixMzgBNdmgN4F/4KbXIMpF7DI89hNTeA==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'ejy8473/GQUnNIwpjTZPFY/SSnP4a4zA16GcLXHjKG0MZZJY4ZoPZ2ixMzgBNdmgN4F/4KbXIMpF7DI89hNTeA==', 'x-amz-request-id': 'MY74Z7CNEWGXC8T0', 'date': 'Tue, 14 May 2024 07:23:02 GMT', 'x-amz-expiration': 'expiry-date="Mon, 20 May 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"7492aca1e8781ada6f0c064874c3550a"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Mon, 20 May 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"7492aca1e8781ada6f0c064874c3550a"', 'ServerSideEncryption': 'AES256'}

In [21]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=split_df2.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/Avinash/Output/CB_data_dump_set2_14-05-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '086TQNBTZ1AXTASV', 'HostId': 'bcrWxTttbymNA8Y1IYiwLJpPIWRTW207IaVV+HgQXhcAP3bPAlHvhb6dK7Mx3PfY6vL21977sOumaQeDiReW6A==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'bcrWxTttbymNA8Y1IYiwLJpPIWRTW207IaVV+HgQXhcAP3bPAlHvhb6dK7Mx3PfY6vL21977sOumaQeDiReW6A==', 'x-amz-request-id': '086TQNBTZ1AXTASV', 'date': 'Tue, 14 May 2024 07:24:40 GMT', 'x-amz-expiration': 'expiry-date="Mon, 20 May 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"eac0b414bb2eacb6e8ca98360d94fd8e"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Mon, 20 May 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"eac0b414bb2eacb6e8ca98360d94fd8e"', 'ServerSideEncryption': 'AES256'}

In [19]:
pce=spark.read.format('delta').load("s3://etl-datahouse/prod/profiles/pce/pce_calc/pce_snapshot/delta").select('mvp_company_name','currentPositions','pastPositions','publicLink', 'fullName', 'gender','ethnicity','functional_workload','business_function','core_Skills','soft_Skills','msa','experience_in_current_company','experience_in_current_job_role','years_of_experience','languages').selectExpr('*','currentPositions[0].companyName as curr_company','currentPositions[0].company_industry as current_industry','currentPositions[0].job_title_lemmatized as current_job_title','currentPositions.job_role[0] as current_job_role','currentPositions[0].msa as current_msa','currentPositions[0].synon_location as current_synon_location','currentPositions[0].i18nStartDate as current_StartDate','currentPositions[0].i18nEndDate as current_EndDate','currentPositions[0].startDateYear as startDateYear','currentPositions[0].endDateYear as endDateYear').drop('currentPositions','deleted') 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
pce.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- mvp_company_name: string (nullable = true)
 |-- pastPositions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- i18nStartDate: string (nullable = true)
 |    |    |-- i18nEndDate: string (nullable = true)
 |    |    |-- companyName: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |    |    |-- location: string (nullable = true)
 |    |    |-- startDateYear: string (nullable = true)
 |    |    |-- startDateMonth: string (nullable = true)
 |    |    |-- endDateYear: string (nullable = true)
 |    |    |-- endDateMonth: string (nullable = true)
 |    |    |-- summary: string (nullable = true)
 |    |    |-- level: string (nullable = true)
 |    |    |-- experience_years: double (nullable = true)
 |    |    |-- company_website: string (nullable = true)
 |    |    |-- company_workforce_size: string (nullable = true)
 |    |    |-- company_industry: string (nullable = true)
 |    |    |-- msa: string (nullable = true)
 

In [21]:
loc_df=spark.read.csv("s3://draup-command-center/temp/cc/Avinash_Draup/Input/Maine_msa.csv",header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:

main_df = pce.join(loc_df, pce['msa'] == loc_df['Location'], "inner").drop('Location')
 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# main_df.write.mode("overwrite").parquet('s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/temp/pce_all_past')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
df4 = spark.read.csv("s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/output/CB_Details_use_set4_02-09-2024.csv",header = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
df1.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['_c0', 'cb_markets', 'company_type', 'permalink', 'founded_on', 'name', 'legal_name', 'description', 'short_description', 'employee_count', 'num_funding_rounds', 'funding_rounds_name', 'cb_url', 'homepage_url', 'domain_url', 'email', 'facebook_url', 'linkedin_url', 'twitter_url', 'logo_url', 'category', 'category_groups', 'company_industry', 'markets', 'organizations_location', 'organizations_msa', 'country_code', 'address', 'city', 'regoin', 'state_code', 'organizations_synon_location', 'stock_exchange_symbol', 'stock_symbol', 'investor_type']

In [66]:
df2 = spark.read.csv("s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/output/CB_Details_use_set2_02-09-2024.csv",header = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
join_df = df1.join(df3, on='permalink', how='inner')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
join_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

218081